In [10]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from itertools import product

# Load data
df = pd.read_csv('日_日交易统计文件.csv')
df = df.rename(columns={
    '交易日收盘价': 'Price',
    '交易日总成交量': 'Volume',
    '交易日总成交金额': 'Amount'
})

TRDC = pd.read_csv('日_股本变动文件.csv')
TRDC = TRDC.iloc[2:]

# Function to create year and month columns
def My_yearmon(Set, YMD):
    Set[YMD] = pd.to_datetime(Set[YMD], format='%Y-%m-%d')
    Set['year'] = Set[YMD].dt.year
    Set['month'] = Set[YMD].dt.month

# Convert dates
My_yearmon(TRDC, '股本变动日期')
My_yearmon(df, '交易日期')

# Create Yearmon column
df['Yearmon'] = pd.to_datetime(df['交易日期']).dt.to_period('M').astype(str)

# Create Yearmon for TRDC and merge to get ID_Mon
TRDC['Yearmon'] = pd.to_datetime(TRDC['股本变动日期']).dt.to_period('M').astype(str)
Mon_list = np.unique(TRDC['Yearmon'])
R2 = pd.DataFrame({'Yearmon': Mon_list, 'ID_Mon': np.arange(len(Mon_list))})
TRDCM = TRDC[['证券代码', 'Yearmon', 'A股流通股数']].drop_duplicates(subset=['证券代码', 'Yearmon'], keep='last').reset_index(drop=True)
TRDCM = TRDCM.merge(R2, on='Yearmon', how='left')

# Create ID_Mon in `df` DataFrame
df = df.merge(R2, on='Yearmon', how='left')

# Now define Stkcd_list and create X1, X2
Stkcd_list = np.unique(df['证券代码'])
X1, X2 = [], []
for x1, x2 in product(Stkcd_list, Mon_list):
    X1.append(x1)
    X2.append(x2)

R1 = pd.DataFrame({'证券代码': X1, 'Yearmon': X2})
R1 = R1.merge(R2, on='Yearmon', how='left')

# Calculate Amount and log volume
AM = df.groupby(['证券代码', 'ID_Mon'])['Amount'].sum().reset_index()
AM['dolvol'] = np.log(AM['Amount'])
AM['ID_Mon_2'] = AM['ID_Mon'] + 2

R1_ = R1.merge(AM[['证券代码', 'ID_Mon_2', 'dolvol']], 
                left_on=['证券代码', 'ID_Mon'], 
                right_on=['证券代码', 'ID_Mon_2'], 
                how='left')

# Calculate Volume
VL = df.groupby(['证券代码', 'ID_Mon'])['Volume'].sum().reset_index()
VL['ID_Mon_1'] = VL['ID_Mon'] + 1
VL['ID_Mon_2'] = VL['ID_Mon'] + 2
VL['ID_Mon_3'] = VL['ID_Mon'] + 3

for n, i in enumerate(['ID_Mon_1', 'ID_Mon_2', 'ID_Mon_3'], start=1):
    R1_ = R1_.merge(VL[['证券代码', i, 'Volume']], 
                     left_on=['证券代码', 'ID_Mon'], 
                     right_on=['证券代码', i], 
                     how='left')
    R1_.rename(columns={'Volume': f'Volume_{n}'}, inplace=True)

# Merge A股流通股数
R1_ = R1_.merge(TRDCM[['证券代码', 'Yearmon', 'A股流通股数']], on=['证券代码', 'Yearmon'], how='left')
R1_.fillna(0, inplace=True)

# Calculate turn
R1_['turn'] = (R1_['Volume_1'] + R1_['Volume_2'] + R1_['Volume_3']) / R1_['A股流通股数']
R1_['turn'].fillna(0, inplace=True)

# Count zero trades
ZE = df[['证券代码', 'ID_Mon', 'Volume']]
ZE['zero'] = np.where(ZE['Volume'] == 0, 1, 0)
ZERO = ZE.groupby(['证券代码', 'ID_Mon'])['zero'].count().reset_index()

ZERO['ID_Mon_1'] = ZERO['ID_Mon'] + 1
R1_2 = R1_.merge(ZERO[['证券代码', 'ID_Mon_1', 'zero']], 
                  left_on=['证券代码', 'ID_Mon'], 
                  right_on=['证券代码', 'ID_Mon_1'], 
                  how='left')

# Calculate standard deviation of Volume and Amount
SD = df.groupby(['证券代码', 'ID_Mon'])['Volume'].std().reset_index()
SD.rename(columns={'Volume': 'std_dolvol'}, inplace=True)

R1_2 = R1_2.merge(SD, on=['证券代码', 'ID_Mon'], how='left')

SD2 = df.groupby(['证券代码', 'ID_Mon'])['Amount'].std().reset_index()
SD2.rename(columns={'Amount': 'std_turn'}, inplace=True)

R1_2 = R1_2.merge(SD2, on=['证券代码', 'ID_Mon'], how='left')

# Calculate flag and filter
R1_2['flag'] = R1_2[['dolvol', 'turn', 'zero', 'std_dolvol', 'std_turn']].sum(axis=1)
R1_2 = R1_2[R1_2['flag'] != 0.0].reset_index(drop=True)

# Save results
R1_2 = R1_2[['证券代码', 'Yearmon', 'dolvol', 'turn', 'zero', 'std_dolvol', 'std_turn']]
R1_2 = R1_2.rename(columns={'Yearmon': '月份'})
R1_2

,证券代码,月份,dolvol,turn,zero,std_dolvol,std_turn
0,1,2003-01,0.000000,0.000000,NaN,1.527816e+07,1.731399e+08
1,1,2003-02,0.000000,inf,8.0,3.479941e+06,3.936750e+07
2,1,2003-03,21.302191,inf,4.0,6.928986e+06,7.806583e+07
3,1,2003-04,19.976613,inf,3.0,2.468480e+07,3.444718e+08
4,1,2003-05,20.460381,inf,17.0,6.704403e+06,8.773737e+07
...,...,...,...,...,...,...,...
676807,920099,2024-10,0.000000,2.511022,4.0,2.610548e+06,1.104131e+08
676808,920099,2024-11,20.733655,inf,4.0,NaN,NaN
676809,920118,2024-09,0.000000,0.000000,NaN,7.503072e+05,1.310361e+07
676810,920118,2024-10,0.000000,0.329354,3.0,1.401359e+06,4.088636e+07


In [11]:
mon_lst = []
for y in range(2000, 2025):
    for m in ['01-31', '02-29' if (y % 4 == 0 and y % 100 != 0) or (y % 400 == 0) else '02-28', '03-31', '04-30', '05-31', '06-30', '07-31', '08-31', '09-30', '10-31', '11-30', '12-31']:
        mon_lst.append(f'{y}-{m}')
mon_df = pd.DataFrame(mon_lst[:-2], columns=['交易月份'])
mon_df['月份'] = mon_df['交易月份'].apply(lambda x: x[:7])
stk_df = pd.read_csv('公司文件.csv')
stk_df['on'] = 0
mon_df['on'] = 0
header = pd.merge(stk_df, mon_df, on=['on'], how='left')[['证券代码', '交易月份', '月份']]
header

,证券代码,交易月份,月份
0,1,2000-01-31,2000-01
1,1,2000-02-29,2000-02
2,1,2000-03-31,2000-03
3,1,2000-04-30,2000-04
4,1,2000-05-31,2000-05
...,...,...,...
1719157,920118,2024-06-30,2024-06
1719158,920118,2024-07-31,2024-07
1719159,920118,2024-08-31,2024-08
1719160,920118,2024-09-30,2024-09


In [12]:
df = pd.merge(header, R1_2, on=['证券代码', '月份'], how='left')

In [13]:
df

,证券代码,交易月份,月份,dolvol,turn,zero,std_dolvol,std_turn
0,1,2000-01-31,2000-01,NaN,NaN,NaN,NaN,NaN
1,1,2000-02-29,2000-02,NaN,NaN,NaN,NaN,NaN
2,1,2000-03-31,2000-03,NaN,NaN,NaN,NaN,NaN
3,1,2000-04-30,2000-04,NaN,NaN,NaN,NaN,NaN
4,1,2000-05-31,2000-05,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1719157,920118,2024-06-30,2024-06,NaN,NaN,NaN,NaN,NaN
1719158,920118,2024-07-31,2024-07,NaN,NaN,NaN,NaN,NaN
1719159,920118,2024-08-31,2024-08,NaN,NaN,NaN,NaN,NaN
1719160,920118,2024-09-30,2024-09,0.0,0.000000,NaN,7.503072e+05,1.310361e+07


In [14]:
df.to_csv("月_27_79_80_85_86.csv", encoding='utf_8_sig',index = False)